In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from pathlib import Path
import yaml

In [1]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path  # Added validation data path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: tuple
    params_learning_rate: float
    params_momentum: float


In [2]:
class ConfigurationManager:
    def __init__(self, config_path="config.yaml"):
        with open(config_path, "r") as file:
            self.config = yaml.safe_load(file)
        
    def get_training_config(self):
        return TrainingConfig(**self.config["training"])

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from pathlib import Path

class Training:
    def __init__(self, config: TrainingConfig, model: nn.Module):
        self.config = config
        self.model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def get_base_model(self):
        # Load the model from the updated base model path (if needed)
        # For PyTorch, typically we load pre-trained weights for transfer learning
        # Assuming model is already initialized
        checkpoint = torch.load(self.config.updated_base_model_path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()  # Set the model to evaluation mode initially

    def get_data_loaders(self):
        # Define the image transforms
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            size=self.config.params_image_size[:-1],  # Assuming image size is in format (H, W, C)
            batch_size=self.config.params_batch_size,
        )

        # Define training transformations
        train_transforms = [transforms.RandomHorizontalFlip(), transforms.RandomRotation(40),
                            transforms.RandomResizedCrop(self.config.params_image_size[0]),
                            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                            transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]

        # Validation transformations
        valid_transforms = [transforms.Resize(self.config.params_image_size[0]),
                            transforms.CenterCrop(self.config.params_image_size[0]),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]

        # If augmentation is enabled, add the transformation; else use just validation transforms
        if self.config.params_is_augmentation:
            train_transforms.append(transforms.RandomHorizontalFlip())
            train_transforms.append(transforms.RandomRotation(40))

        # Apply transformations
        train_transform = transforms.Compose(train_transforms)
        valid_transform = transforms.Compose(valid_transforms)

        # Load datasets
        train_dataset = datasets.ImageFolder(self.config.training_data, transform=train_transform)
        valid_dataset = datasets.ImageFolder(self.config.validation_data, transform=valid_transform)

        # Create DataLoader instances
        train_loader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=self.config.params_batch_size, shuffle=False)

        return train_loader, valid_loader

    def save_model(self, path: Path):
        torch.save({
            'model_state_dict': self.model.state_dict(),
        }, path)

    def train(self):
        # Get data loaders
        train_loader, valid_loader = self.get_data_loaders()

        # Set loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        # Training loop
        self.model.train()
        for epoch in range(self.config.params_epochs):
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                # Track statistics
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print(f'Epoch [{epoch+1}/{self.config.params_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

            # Validation loop
            self.model.eval()  # Set model to evaluation mode
            with torch.no_grad():
                correct = 0
                total = 0
                for inputs, labels in valid_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)

                    outputs = self.model(inputs)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print(f'Validation Accuracy: {100 * correct / total:.2f}%')

            # Save the model after each epoch
            self.save_model(self.config.trained_model_path)

        print("Training Finished!")


In [4]:
try:
    # Initialize ConfigurationManager to read configuration files
    config = ConfigurationManager()
    
    # Get the training configuration
    training_config = config.get_training_config()
    
    # Initialize the model (for instance, a pre-trained model or a custom CNN)
    # Ensure that your model class is compatible with the config
    # For example, a simple CNN model could be used, or you could load a pre-trained model
    model = YourModelClass(config)  # Replace with your actual model class (e.g., CNN)

    # Initialize the Training class with the configuration and model
    training = Training(config=training_config, model=model)
    
    # Load the base model (e.g., pre-trained weights)
    training.get_base_model()
    
    # Get the data generators for training and validation
    training.train_valid_generator()
    
    # Start training the model
    training.train()

except Exception as e:
    print(f"An error occurred: {e}")
    raise e


An error occurred: [Errno 2] No such file or directory: 'config.yaml'


FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'